ROOT Version
====

Extracting a signal from two datasets.
---
Below we simulate two experiments.

Experiment1:
- has a gaussian signal
- and a falling background that goes as $exp^{-x/\lambda}$

Experinemt2:
- has the same gaussian signal component
- and a background that goes as $x^n$, where $n$<0

In [5]:
!pip install iminuit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.2/448.2 kB 30.7 MB/s eta 0:00:00


In [4]:
import ROOT as r
import numpy as np
from iminuit import Minuit



In [6]:
tfsig=r.TF1("tfsig","exp(-0.5*(x-[0])*(x-[0])/[1]/[1])",25,125)
tfsig.SetParameters(75,4.5)

def experiment1():
    S_over_N = 0.08
    ndata=2700
    lam=20
    nbins=50
    xrange=(30,100)
    background = r.TF1("back1","exp(-x/[0])",xrange[0],xrange[1])
    background.SetParameter(0,lam)
    hist = r.TH1F("hexp1","Experiment1;x;frequency",nbins,xrange[0],xrange[1])
    nsig=int(ndata*S_over_N)
    nbkg=ndata-nsig
    hist.FillRandom("tfsig",nsig)
    hist.FillRandom("back1",nbkg)
    return hist

def experiment2():
    S_over_N = 0.12
    ndata=2500
    n=-2.2
    xrange=(50,100)
    nbins=50
    background = r.TF1("back2","pow(x,[0])",xrange[0],xrange[1])
    background.SetParameter(0,n)
    hist = r.TH1F("hexp2","Experiment2;x;frequency",nbins,xrange[0],xrange[1])
    nsig=int(ndata*S_over_N)
    nbkg=ndata-nsig
    hist.FillRandom("tfsig",nsig)
    hist.FillRandom("back2",nbkg)
    return hist

Here we run the two experiments and get the results.  We will interpret these as follows:

- The experiments are independent
- They measure the same signal process
- They have different backgrounds to the signal measurement

In [11]:
tc=r.TCanvas()
tc.Divide(2,1)
h1=experiment1()
h2=experiment2()
tc.cd(1)
h1.Draw("e")
tc.cd(2)
h2.Draw("e")
tc.Draw()

Error in callback <bound method CaptureDrawnPrimitives._post_execute of <JupyROOT.helpers.utils.CaptureDrawnPrimitives object at 0x7b13082421e0>> (for post_execute):


Warning in <TROOT::Append>: Replacing existing TH1: hexp1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hexp2 (Potential memory leak).


AttributeError: <class cppyy.gbl.TWebCanvas at 0x14884d00> has no attribute 'CreateCanvasJSON'. Full details:
  type object 'TWebCanvas' has no attribute 'CreateCanvasJSON'
  'TWebCanvas::CreateCanvasJSON' is not a known C++ class
  'CreateCanvasJSON' is not a known C++ template
  'CreateCanvasJSON' is not a known C++ enum

Here we save the results of the experiments:

In [12]:
tf=r.TFile("experiments.root","recreate")
h1.Write()
h2.Write()
tf.Close()

And here's an example of reading them back from the TFile

Below we use DrawCopy instead of Draw, so we can close the file (which deletes the histogram from memory) without deleting the drawing.

In [2]:
tf=r.TFile("experiments.root")
h1=tf.Get("hexp1")
h2=tf.Get("hexp2")
tc.cd(1)
h1.DrawCopy("e")
tc.cd(2)
h2.DrawCopy("e")
tc.Draw()
tf.Close()

NameError: name 'r' is not defined

You job for this project will be to develop a simultaneous fit for the two histograms using minuit.  See this week's exercise description for more details.

In [5]:
tf=r.TFile("experiments.root")
h1=tf.Get("hexp1")
h2=tf.Get("hexp2")

nbins1 = h1.GetNbinsX()
x1 = np.array([h1.GetBinCenter(i) for i in range(1, nbins1+1)])
y1 = np.array([h1.GetBinContent(i) for i in range(1, nbins1+1)])
err1 = np.array([h1.GetBinError(i) for i in range(1, nbins1+1)])

nbins2 = h2.GetNbinsX()
x2 = np.array([h2.GetBinCenter(i) for i in range(1, nbins2+1)])
y2 = np.array([h2.GetBinContent(i) for i in range(1, nbins2+1)])
err2 = np.array([h2.GetBinError(i) for i in range(1, nbins2+1)])

def gaussian(x, mu, sigma):
    return np.exp(-0.5 * ((x-mu)/sigma)**2) / (np.sqrt(2*np.pi)*sigma)
def model1(x, mu, sigma, S1, A1, lam1):
  return S1 * gaussian(x, mu, sigma) + A1 * np.exp(-x/lam1)
def model2(x, mu, sigma, S2, A2, n2):
    return S2 * gaussian(x, mu, sigma) + A2 * (x**n2)

#total chi^2 = chi_1^2 + chi_2^2
def chi2_total(mu, sigma, S1, A1, lam1, S2, A2, n2):
    y1_exp = model1(x1, mu, sigma, S1, A1, lam1)
    y2_exp = model2(x2, mu, sigma, S2, A2, n2)
    chi2_1 = np.sum(((y1 - y1_exp)/err1)**2)
    chi2_2 = np.sum(((y2 - y2_exp)/err2)**2)
    return chi2_1 + chi2_2

m = Minuit(
    chi2_total,
    mu=60, sigma=10,
    S1=200, A1=200, lam1=30,
    S2=100, A2=50, n2=-1
)
m.errordef = 1
m.migrad()

print(m.values)
print(m.errors)

<ValueView mu=62.55312968901014 sigma=16.48277042219576 S1=1158.765296205337 A1=2346.2943534791507 lam1=12.29080725634027 S2=1560.0297895030246 A2=5344.727442142331 n2=-1.2642589596723992>
<ErrorView mu=1.7160179180870252 sigma=1.163839337619769 S1=149.22797324813078 A1=299.7822895266862 lam1=0.5811093729994777 S2=267.9886066640277 A2=1259.7398251761713 n2=0.06330885418491117>
